# Up and Running with TensorFlow

Its basic principle is simple: you first define in Python a graph of computations to perform and then TensorFlow takes that graph and runs it efficiently using optimized C++ code.
It is possible to break up the graph into several chunks and run them in parallel across multiple CPUs or GPUs.

# Create and Run a Graph in a Session

In [1]:
import tensorflow as tf

x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

In [2]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)

result = sess.run(f)

print(result)

42


In [3]:
sess.close()

In [5]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    
print(result)

42


In [7]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run() # initialize all the variables
    result = f.eval()
    print(result)

42


# Managing Graphs

In [9]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [10]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [11]:
x2.graph is graph

True

In [12]:
x2.graph is tf.get_default_graph()

False

# Lifecycle of a Node Value

In [13]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

In [15]:
with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [16]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


# Linear Regression with TensorFlow

In [18]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]


X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')

XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)), XT), y)

In [19]:
with tf.Session() as sess:
    theta_value = theta.eval()
    print(theta_value)

[[ -3.74651413e+01]
 [  4.35734153e-01]
 [  9.33829229e-03]
 [ -1.06622010e-01]
 [  6.44106984e-01]
 [ -4.25131839e-06]
 [ -3.77322501e-03]
 [ -4.26648885e-01]
 [ -4.40514028e-01]]


# Implementing Gradient Descent

In [172]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0), name='theta')

y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y

mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error)

training_op = tf.assign(theta, theta - learning_rate*gradients)

init = tf.global_variables_initializer()

In [173]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch%500 == 0:
            print('Epoch:', epoch, 'MSE:', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
    print(best_theta)

Epoch: 0 MSE: 9.17788
Epoch: 500 MSE: 0.542486
[[ 2.06855226]
 [ 0.84509295]
 [ 0.14217678]
 [-0.25637344]
 [ 0.28124443]
 [ 0.00381733]
 [-0.04163765]
 [-0.71458662]
 [-0.68518585]]


# Using autodiff
Can you find the equation to compute the partial derivatives of the following function?

In [174]:
def my_func(a, b):
    z = 0
    for i in range(100):
        z = a*np.cos(z + i) + z*np.sin(b - i)
    return z

In [175]:
my_func(0.2, 0.3)

-0.21253923284754916

In [176]:
a = tf.Variable(0.2, name='a')
b = tf.Variable(0.3, name='b')
z = tf.Variable(0.0, name='z')

for i in range(100):
    z = a*tf.cos(z + i) + z*tf.sin(b - i)
    
grads = tf.gradients(z, [a, b])
init = tf.global_variables_initializer()

In [177]:
with tf.Session() as sess:
    init.run()
    print(z.eval())
    print(sess.run(grads))

-0.212537
[-1.1388494, 0.19671395]


In [178]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

gradients = tf.gradients(mse, [theta])[0] # changed!

In [179]:
training_op = tf.assign(theta, theta - learning_rate*gradients)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch%100 == 0:
            print('Epoch:', epoch, 'MSE:', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
    print(best_theta)

Epoch: 0 MSE: 9.24596
Epoch: 100 MSE: 0.690023
Epoch: 200 MSE: 0.536117
Epoch: 300 MSE: 0.530664
Epoch: 400 MSE: 0.528915
Epoch: 500 MSE: 0.527705
Epoch: 600 MSE: 0.526823
Epoch: 700 MSE: 0.526177
Epoch: 800 MSE: 0.525702
Epoch: 900 MSE: 0.525353
[[ 2.06855226]
 [ 0.79884017]
 [ 0.12230985]
 [-0.18992476]
 [ 0.23550563]
 [-0.00280943]
 [-0.0389709 ]
 [-0.90284604]
 [-0.86907482]]


# Using a GradientDescentOptimizer¶

In [180]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

In [181]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

In [182]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch%100 == 0:
            print('Epoch:', epoch, 'MSE:', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
    print(best_theta)

Epoch: 0 MSE: 12.3797
Epoch: 100 MSE: 0.751954
Epoch: 200 MSE: 0.583298
Epoch: 300 MSE: 0.566658
Epoch: 400 MSE: 0.5562
Epoch: 500 MSE: 0.548459
Epoch: 600 MSE: 0.542686
Epoch: 700 MSE: 0.538362
Epoch: 800 MSE: 0.535112
Epoch: 900 MSE: 0.532657
[[ 2.06855226]
 [ 0.86329037]
 [ 0.14670542]
 [-0.28866675]
 [ 0.30703038]
 [ 0.00522901]
 [-0.04240884]
 [-0.66449171]
 [-0.63710785]]


# Using a Momentum optimizer

In [183]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,
                                      momentum=0.9)
training_op = optimizer.minimize(mse)


init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch%100 == 0:
            print('Epoch:', epoch, 'MSE:', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
    print(best_theta)

Epoch: 0 MSE: 6.35652
Epoch: 100 MSE: 0.556764
Epoch: 200 MSE: 0.528036
Epoch: 300 MSE: 0.524804
Epoch: 400 MSE: 0.524385
Epoch: 500 MSE: 0.52433
Epoch: 600 MSE: 0.524322
Epoch: 700 MSE: 0.524321
Epoch: 800 MSE: 0.524321
Epoch: 900 MSE: 0.524321
[[ 2.06855774]
 [ 0.82964069]
 [ 0.11875564]
 [-0.26556748]
 [ 0.30572984]
 [-0.00450184]
 [-0.03932706]
 [-0.89983654]
 [-0.87049437]]


# Feeding Data to the Training Algorithm

In [185]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5

with tf.Session() as sess:
    b1 = B.eval(feed_dict={A: [[1,2,3]]})
    b2 = B.eval(feed_dict={A: [[4,5,6], [7,8,9]]})
    
print(b1)
print(b2)

[[ 6.  7.  8.]]
[[  9.  10.  11.]
 [ 12.  13.  14.]]


## Mini-Batch Gradient Descent

In [186]:
m, n = housing.data.shape

X = tf.placeholder(tf.float32, shape=(None, n + 1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')


batch_size = 128
n_batches = int(np.ceil(m / batch_size))

In [187]:
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
                                    
training_op = optimizer.minimize(mse)


init = tf.global_variables_initializer()

In [188]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  
    indices = np.random.randint(m, size=batch_size)  
    X_batch = scaled_housing_data_plus_bias[indices] 
    y_batch = housing.target.reshape(-1, 1)[indices] 
    return X_batch, y_batch

In [189]:
n_epochs = 10
learning_rate = 0.01

In [190]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

In [191]:
print('Best theta\n', best_theta)

Best theta
 [[ 2.06643343]
 [ 0.79742289]
 [ 0.129503  ]
 [-0.16942203]
 [ 0.27013609]
 [-0.00423199]
 [-0.04918712]
 [-0.84386688]
 [-0.79764754]]


# Saving and restoring a model¶

In [192]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)


init = tf.global_variables_initializer()
# saver
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch%100 == 0:
            print('Epoch:', epoch, 'MSE:', mse.eval())
            #save_path = saver.save(sess, "/tmp/my_model.ckpt")
        sess.run(training_op)
    best_theta = theta.eval()
    save_path = saver.save(sess, './tmp/my_model_final.ckpt')
    
print(best_theta)

Epoch: 0 MSE: 7.10993
Epoch: 100 MSE: 0.647945
Epoch: 200 MSE: 0.534951
Epoch: 300 MSE: 0.530466
Epoch: 400 MSE: 0.528753
Epoch: 500 MSE: 0.527551
Epoch: 600 MSE: 0.526679
Epoch: 700 MSE: 0.526045
Epoch: 800 MSE: 0.525585
Epoch: 900 MSE: 0.525249
[[  2.06855249e+00]
 [  8.26840758e-01]
 [  1.27716556e-01]
 [ -2.42535949e-01]
 [  2.78882653e-01]
 [ -1.20536773e-03]
 [ -4.00254577e-02]
 [ -8.37146401e-01]
 [ -8.06620181e-01]]


In [193]:
with tf.Session() as sess:
    saver.restore(sess, "./tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval()
best_theta_restored

INFO:tensorflow:Restoring parameters from ./tmp/my_model_final.ckpt


array([[  2.06855249e+00],
       [  8.26840758e-01],
       [  1.27716556e-01],
       [ -2.42535949e-01],
       [  2.78882653e-01],
       [ -1.20536773e-03],
       [ -4.00254577e-02],
       [ -8.37146401e-01],
       [ -8.06620181e-01]], dtype=float32)

In [194]:
tf.reset_default_graph()

saver = tf.train.import_meta_graph("./tmp/my_model_final.ckpt.meta")  # this loads the graph structure
theta = tf.get_default_graph().get_tensor_by_name("theta:0") # not shown in the book

with tf.Session() as sess:
    saver.restore(sess, "./tmp/my_model_final.ckpt")  # this restores the graph's state
    best_theta_restored = theta.eval() 

INFO:tensorflow:Restoring parameters from ./tmp/my_model_final.ckpt


# Visualizing the Graph

In [195]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b"<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [196]:
show_graph(tf.get_default_graph())

# Using TensorBoard¶

In [203]:
tf.reset_default_graph()

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [204]:
n_epochs = 5000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=16), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [205]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [206]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [207]:
with tf.Session() as sess:                                                      
    sess.run(init)                                                      

    for epoch in range(n_epochs):                                        
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()                                             

In [208]:
file_writer.close()
best_theta

array([[ 2.07029438],
       [ 0.85185266],
       [ 0.1188163 ],
       [-0.29350281],
       [ 0.37137547],
       [ 0.00318078],
       [-0.01125147],
       [-0.87467641],
       [-0.842489  ]], dtype=float32)

In [ ]:
# source env/bin/activate
# tensorboard --logdir tf_logs/